# Atividade 3

Faça tudo via dbt. 

Envia como resposta:
- A query SQL
- Comando utilizado via terminal no `dbt`
- Um print do resultado no próprio notebook, para isso é necessário criar uma pasta no repositório onde serão armazenadas as imagens, e para inserir no notebook é necessário criar uma célula `markdown` e inserir o seguinte comando: 

```markdown
![Nome da Foto](path/para/imagem)
```

## 1. Crie uma tabela `orders_by_year` a partir de `orders` que agrupa a quantidade de pedidos por ano, considerando a coluna `order_approved_at`

### 1.1 Query:
    
```sql
{{ config(materialized='table') }}

SELECT
    EXTRACT(YEAR from date(order_approved_at)) AS year, 
    count(order_id) AS num_orders 
FROM 
    `dbt_shop.orders` 
GROUP BY 
    1
ORDER BY
    1
```

### 1.2 Terminal: 
```dbt run --models atividade03.orders_by_year```

### 1.3 - Print Terminal:
![orders_by_year_terminal.png](print_resultados_atividade03/orders_by_year_terminal.png)

### 1.4 No GCP:
![orders_by_year_gcp.png](print_resultados_atividade03/orders_by_year_gcp.png)

## 2. Crie uma `view` que agrega os totais de pedidos por estado. Considere as tabelas _customers_ para obter os estados (coluna _customer_state_) e _orders_ para contabilizar a quantidade de pedidos.

### 2.1 Query:

```sql
{{ config(materialized='view') }}

SELECT
    A.customer_state, 
    count(B.order_id) AS num_orders 
FROM 
    `dbt_shop.customers`  AS A
LEFT JOIN
    `dbt_shop.orders`     AS B
ON
    A.customer_id =  B.customer_id
GROUP BY 
    1
ORDER BY
    1
```

### 2.2 Terminal: 
```dbt run --models atividade03.orders_by_state```

### 2.3 Print Terminal:
![orders_by_state_terminal.png](print_resultados_atividade03/orders_by_state_terminal.png)

### 2.4 No GCP:
![orders_by_state_gcp.png](print_resultados_atividade03/orders_by_state_gcp.png)

## 3. De acordo com o resultado do exercício 2 retorne apenas os estados que tem mais de 5000 pedidos, esse número deve ser passado via variável `number_of_orders` do `dbt`.

### 3.1 Query:

```sql
{{ config(materialized='view') }}

SELECT
    *
FROM 
    `dbt-shop-jonatas.dataset_atividade03.orders_by_state`  AS A
WHERE 
     num_orders >= {{ var("number_of_orders") }}
ORDER BY
    1
```

### 3.2 Terminal: 
```dbt run --models atividade03.orders_by_state_5000 --vars "{'number_of_orders': 5000}"```

### 3.3 Print Terminal:
![orders_by_state_5000_terminal_v2.png](print_resultados_atividade03/orders_by_state_5000_terminal_v2.png)

### 3.4 No GCP:
![orders_by_state_5000_gcp.png](print_resultados_atividade03/orders_by_state_5000_gcp.png)

## 4. Crie uma `view` chamada `customers_by_region` que utiliza os dados da tabela `customers`. A visão deve realizar uma agregação por região do Brasil (Norte, Nordeste, Sudeste, Centro Oeste e Sul) e contar o número de clientes em cada uma delas.

### 4.1 Query:

```sql
{{ config(materialized='view') }}

SELECT 
    CASE WHEN customer_state IN ("DF","GO","MS","MT") THEN "Centro-Oeste"
         WHEN customer_state IN ("AL","BA","CE","MA","PB","PE","PI","RN","SE") THEN "Nordeste"
         WHEN customer_state IN ("AC","AM","AP","PA","RO","RR","TO") THEN "Norte"
         WHEN customer_state IN ("ES","MG","RJ","SP") THEN "Sudeste"
         WHEN customer_state IN ("PR","RS","SC") THEN "Sul"
         ELSE "Não Identificado" END AS customer_region,
    count(customer_id) as customer_quantity

FROM 
    `dbt_shop.customers`  
GROUP BY
    1
```

### 4.2 Terminal: 
```dbt run --models atividade03.customers_by_region```


### 4.3 Print Terminal:
![customers_by_region_terminal.png](print_resultados_atividade03/customers_by_region_terminal.png)

### 4.4 No GCP:
![customers_by_region_gcp.png](print_resultados_atividade03/customers_by_region_gcp.png)

## 5. Crie uma tabela `items_costs` que contenha o preço total de um item (soma do valor do produto com o frete).

### 5.1 Query:

```sql
{{ config(materialized='table') }}

SELECT
    *,
    price + freight_value as total_cost
FROM 
    `dbt_shop.items` 
```

### 5.2 Terminal:
```dbt run --models atividade03.items_costs```

### 5.3 Print Terminal:
![items_costs_terminal.png](print_resultados_atividade03/items_costs_terminal.png)

### 5.4 No GCP:
![items_costs_gcp.png](print_resultados_atividade03/items_costs_gcp.png)

## 6. Crie uma `view` com o nome `orders_shipped_costs`, você deve calcular o preço total do pedido somado com o frete, apenas para pedidos enviado (shipped).

### 6.1 Query:

```sql
{{ config(materialized='view') }}

SELECT
    A.order_id,
    B.total_cost
FROM 
    `dbt_shop.orders` AS A
INNER JOIN
  (
    SELECT 
      order_id,
      SUM(price + freight_value) AS total_cost
    FROM
      `dbt_shop.items`
    GROUP BY  
      1
  ) AS B
ON
  A.order_id = B.order_id AND A.order_status = "shipped"
```

### 6.2 Terminal: 
``dbt run --models atividade03.orders_shipped_costs``

### 6.3 Print Terminal:
![orders_shipped_costs_terminal.png](print_resultados_atividade03/orders_shipped_costs_terminal.png)

### 6.4 No GCP:
![orders_shipped_costs_gcp.png](print_resultados_atividade03/orders_shipped_costs_gcp.png)